# Red Wine Quality
Coursera Advanced Data Science Capstone Project

## Extract, Transform, Load 
In the exploration, it was determined that the dataset can be held in memory (i.e. no need for parallelized libraries), and that the 11 features all are numerical. The target variable is a categorical value.

The preproccessing needed thus is to 
1. load the data, 
2. ensure data type conformity,
3. normalize the 11 features,
4. one-hot encode the target

### 1 - Load the data


In [2]:
# Import data preprocessing libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler, Binarizer


In [3]:
# Import dataset from IBM ObjectStorage as Pandas Dataframe
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ab7ec63426ba4703bfcdc7e2b7592286 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XX',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_ab7ec63426ba4703bfcdc7e2b7592286.get_object(Bucket='ibmadvancedcapstone-donotdelete-pr-6kout1ocpnvgg4',Key='winequality-red.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)

### 2 - Ensure data type conformity

Let's check the size of the data, as well as the format of each of the columns

In [4]:
print("The dataset contains {} rows with {} features each".format(df.shape[0], df.shape[1]))
df.head()

The dataset contains 1599 rows with 12 features each


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
# Check dataframe datatypes
df.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

As we can see, we have a dataset of almost 1600 entries (12 features each), all with the correct float/integer datatypes.

### 3 - Feature normalization (& outlier removal)

Let's get rid of some of the most obvious outlier samples by clipping the dataset

In [6]:
features = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]
target = "quality"

df["fixed acidity"] = df["fixed acidity"].clip(upper=14)
df["volatile acidity"] = df["volatile acidity"].clip(upper=12)
df["citric acid"] = df["citric acid"].clip(upper=0.8)
df["residual sugar"] = df["residual sugar"].clip(upper=5)
df["chlorides"] = df["chlorides"].clip(upper=0.2)
df["free sulfur dioxide"] = df["free sulfur dioxide"].clip(upper=45)
df["total sulfur dioxide"] = df["total sulfur dioxide"].clip(upper=150)
df["density"] = df["density"].clip(upper=1.002)
df["pH"] = df["pH"].clip(upper=3.8, lower=2.9)
df["sulphates"] = df["sulphates"].clip(upper=1.12)
df["alcohol"] = df["alcohol"].clip(upper=14)

In [7]:
# Normalize features
data_scaled = StandardScaler().fit_transform(df[features].values)
df_scaled = pd.DataFrame(data_scaled, index=df.index, columns=features)
df_scaled.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.532007,0.961877,-1.393725,-0.606467,-0.297244,-0.475048,-0.384530,0.568874,1.308262,-0.635523,-0.961640
1,-0.298960,1.967442,-1.393725,0.214552,0.501811,0.927540,0.649012,0.032479,-0.728588,0.182345,-0.585419
2,-0.298960,1.297065,-1.187895,-0.137313,0.283887,-0.074308,0.241859,0.139758,-0.334359,-0.022122,-0.585419
3,1.681938,-1.384443,1.487886,-0.606467,-0.333564,0.126061,0.429775,0.676153,-0.991407,-0.499212,-0.585419
4,-0.532007,0.961877,-1.393725,-0.606467,-0.297244,-0.475048,-0.384530,0.568874,1.308262,-0.635523,-0.961640


In [8]:
# One-hot encoding of target
data_onehot = Binarizer(threshold=6).fit_transform(df[target].values.reshape(-1,1))
df_onehot = pd.DataFrame(data_onehot, index=df.index, columns=[target])
df_onehot.head()

,quality
0,0
1,0
2,0
3,0
4,0


In [10]:
# Re-merge dataset into new dataframe

df_processed = df_scaled.join(df_onehot)
df_processed.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,-0.532007,0.961877,-1.393725,-0.606467,-0.297244,-0.475048,-0.384530,0.568874,1.308262,-0.635523,-0.961640,0
1,-0.298960,1.967442,-1.393725,0.214552,0.501811,0.927540,0.649012,0.032479,-0.728588,0.182345,-0.585419,0
2,-0.298960,1.297065,-1.187895,-0.137313,0.283887,-0.074308,0.241859,0.139758,-0.334359,-0.022122,-0.585419,0
3,1.681938,-1.384443,1.487886,-0.606467,-0.333564,0.126061,0.429775,0.676153,-0.991407,-0.499212,-0.585419,0
4,-0.532007,0.961877,-1.393725,-0.606467,-0.297244,-0.475048,-0.384530,0.568874,1.308262,-0.635523,-0.961640,0


In [21]:
# @hidden_cell

!pip install pyspark==2.4.5
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()


from project_lib import Project
project = Project(sc, "6e4b5fe0-b485-433d-be79-17508b962f74","p-a47471a031f43beb66e713d119d65d509c07aa23")
project.save_data(file_name = "winequality-red-processed.csv",data = df_processed.to_csv(index=False))

     |████████████████████████████████| 217.8MB 16.0MB/s eta 0:00:0116.9MB/s eta 0:00:08/s eta 0:00:10�██████████          | 150.3MB 9.3MB/s eta 0:00:08     |██████████████████████▍         | 152.3MB 9.3MB/s eta 0:00:08
     |████████████████████████████████| 204kB 47.8MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


{'file_name': 'winequality-red-processed.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmadvancedcapstone-donotdelete-pr-6kout1ocpnvgg4',
 'asset_id': 'a73ed53a-a00a-4425-a96f-97eae5fa0c07'}